In [1]:
# what vars do i want to tune (for EV)
# l1, l2, l3, threshold
def golemEV(lambda_1, lambda_2, lambda_3, thresh):
    from GOLEMTS.model import GolemTS2
    from GOLEMTS.generate_data import SyntheticDataset
    import GOLEMTS.trainer as trainer
    import numpy as np
    d = 10
    p = 1
    n = 1000

    # A_init = np.random.randn((p+1) *d, d)
    dag_obj = SyntheticDataset(n=n, d=d, p=p, B_scale=1.0, graph_type='ER', degree=2, A_scale=1.0, noise_type='EV')
    model = GolemTS2(n=n, d=d, p=p, Y=dag_obj.Y, lambda_1=lambda_1, lambda_2=lambda_2, A_init=None, ev=True, lr=3e-3, lambda_3=lambda_3)
    likes, evs = trainer.train(model, dag_obj.Y, epochs=40_000, warmup_epochs=0, log=False)
    model_B = model.B.detach().numpy()
    model_B[np.abs(model_B) < thresh] = 0
    model_A = model_B[:, :d]
    return model_A, dag_obj.A







In [2]:
def golemEV_grid(lambda_1_l, lambda_2_l, lambda_3_l, thresh_l):
    out_list = []
    for lambda_1 in lambda_1_l:
        for lambda_2 in lambda_2_l:
            for lambda_3 in lambda_3_l:
                for thresh in thresh_l:
                    out_list.append({
                        'lambda_1': lambda_1,
                        'lambda_2': lambda_2,
                        'lambda_3': lambda_3,
                        'thresh': thresh,
                    })
    return out_list


In [3]:
grid = golemEV_grid(
    lambda_1_l=[1e-3, 1e-2, 1e-1],
    lambda_2_l=[0.1, 1, 5, 10],
    lambda_3_l=[0.1, 1, 5, 10],
    thresh_l=[0.01, 0.1, 0.2, 0.3]
)

In [4]:
import grid_search

In [5]:
grid_search.perform_grid_search(grid, model_func=golemEV, output_dir='golemEV03.jsonl')

{'fdr': 0.13333333333333333, 'tpr': 0.4482758620689655, 'fpr': 0.030303030303030304, 'shd': 34, 'pred_size': 30, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.01}
{'fdr': 0.08333333333333333, 'tpr': 0.5409836065573771, 'fpr': 0.023255813953488372, 'shd': 29, 'pred_size': 36, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.1}
{'fdr': 0.18181818181818182, 'tpr': 0.54, 'fpr': 0.04285714285714286, 'shd': 27, 'pred_size': 33, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.2}
{'fdr': 0.1388888888888889, 'tpr': 0.6888888888888889, 'fpr': 0.034482758620689655, 'shd': 17, 'pred_size': 36, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.3}
{'fdr': 0.046511627906976744, 'tpr': 0.8913043478260869, 'fpr': 0.013888888888888888, 'shd': 7, 'pred_size': 43, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 1, 'thresh': 0.01}
{'fdr': 0.0, 'tpr': 0.8541666666666666, 'fpr': 0.0, 'shd': 7, 'pred_size': 41, 'lambda_1': 0.001, 'lambda_2': 0

In [6]:
# what vars do i want to tune (for EV)
# l1, l2, l3, threshold
def golemNV(lambda_1, lambda_2, lambda_3, thresh):
    from GOLEMTS.model import GolemTS2
    from GOLEMTS.generate_data import SyntheticDataset
    import GOLEMTS.trainer as trainer
    import numpy as np
    d = 10
    p = 1
    n = 1000

    # A_init = np.random.randn((p+1) *d, d)
    dag_obj = SyntheticDataset(n=n, d=d, p=p, B_scale=1.0, graph_type='ER', degree=2, A_scale=1.0, noise_type='NV')
    model = GolemTS2(n=n, d=d, p=p, Y=dag_obj.Y, lambda_1=lambda_1, lambda_2=lambda_2, A_init=None, ev=False, lr=3e-3, lambda_3=lambda_3)
    likes, evs = trainer.train(model, dag_obj.Y, epochs=40_000, warmup_epochs=10_000, log=False)
    model_B = model.B.detach().numpy()
    model_B[np.abs(model_B) < thresh] = 0
    model_A = model_B[:, :d]
    return model_A, dag_obj.A







In [7]:
grid_search.perform_grid_search(grid, model_func=golemEV, output_dir='golemNV01.jsonl', thresh=0.01)

{'fdr': 0.12121212121212122, 'tpr': 0.47540983606557374, 'fpr': 0.031007751937984496, 'shd': 35, 'pred_size': 33, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.01}
{'fdr': 0.13157894736842105, 'tpr': 0.55, 'fpr': 0.038461538461538464, 'shd': 30, 'pred_size': 38, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.1}
{'fdr': 0.05263157894736842, 'tpr': 0.7058823529411765, 'fpr': 0.014388489208633094, 'shd': 16, 'pred_size': 38, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.2}
{'fdr': 0.17073170731707318, 'tpr': 0.576271186440678, 'fpr': 0.05343511450381679, 'shd': 30, 'pred_size': 41, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 0.1, 'thresh': 0.3}
{'fdr': 0.0, 'tpr': 0.7288135593220338, 'fpr': 0.0, 'shd': 16, 'pred_size': 43, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 1, 'thresh': 0.01}
{'fdr': 0.0, 'tpr': 0.8571428571428571, 'fpr': 0.0, 'shd': 6, 'pred_size': 36, 'lambda_1': 0.001, 'lambda_2': 0.1, 'lambda_3': 1, 'thresh': 0.1